In [253]:
import xlwings as xw
import pandas as pd
import json

In [228]:
wb = xw.Book('Matrix.xlsx')
#wb = xw.Book(r'C:\Users\Porter\OneDrive - Danmarks Tekniske Universitet\Christian Bertram\MDM.xlsx')

In [229]:
totalmdm = wb.sheets['Sheet1']

In [230]:
mdmdata = totalmdm.range('A1:BY78').value

In [231]:
#get nodes
mdmnodes = mdmdata[2] #first row
for node in mdmnodes:
    if not node:
        print node

None
None
None


In [232]:
#assign levels to nodes
newnodes = []
for index, mdmnode in enumerate(mdmnodes):
    node = {}
    if (mdmnode != 'None') and (index != 0) and mdmnode:
        node["name"] = mdmnode
        node["level"] = mdmdata[1][index]
        node["index"] = index - 3
        node["children"] = []
        #print mdmnode, node["level"]
        newnodes.append(node)

In [233]:
#map node names to indices
nodesdict = {}
for node in newnodes:
    nodesdict[node["name"]] = node["index"]
nodesdict

{u'Anti-shake system': 54,
 u'Anti-theft system': 43,
 u'Arrival notification': 58,
 u'Automatic routing': 24,
 u'Battery Lipo 2S-7.4V': 64,
 u'Beer opener': 56,
 u'Builder': 17,
 u'Capacity 6 bottles': 45,
 u'Changeable battery': 49,
 u'Charger': 68,
 u'Charging Pluto': 7,
 u'Climbing up small slope': 44,
 u'Compartment for Beverages': 73,
 u'Control System': 70,
 u'Cooling system for beverages': 30,
 u'Cost per delivery 2.5 euro': 48,
 u'Data transfer to Pluto': 38,
 u'Day and night': 22,
 u'Demo mode': 59,
 u'Demonstration outside': 9,
 u'Demonstration, Show inside': 12,
 u'Different drinks delivery': 53,
 u'Disassembled transport': 8,
 u'Driving outdoor': 31,
 u'Driving unloaded back': 5,
 u'Driving with load': 3,
 u'Electronic Driver': 67,
 u'Error handling': 41,
 u'Error handling case': 11,
 u'Frame': 62,
 u'General misuse': 4,
 u'HUI': 61,
 u'IPEK /ID': 18,
 u'Indoor navigation and delivery': 46,
 u'Live position tracking': 55,
 u'Loading of beverages': 1,
 u'Lock/unlock beer co

In [234]:
def attachChildren(startindex):
    print "attachChildren of node: ", newnodes[startindex]['name'] 
    #startindex = node.index
    startnode = newnodes[startindex]
    children = findChildren(startindex)
    print "returned children are: ", children
    startnode["children"].append(children)
    for child in children:
        attachChildren(nodesdict[child])
    return

In [235]:
#find immediate children
#iterate through next nodes / add to resultarray if node.level is next to startnode level
#if node.level equals start node level, we exit the function
def findChildren(startindex):
    resultarray = []
    print "Returning children for node ", newnodes[startindex] 
    startlevel = newnodes[startindex]["level"]
    for node in newnodes[startindex+1:]:
        print "checking node %s with index %d and level %d and startindex %d" % (node["name"], node["index"], node["level"], startindex)  
        if node["level"] == startlevel + 1:
            resultarray.append(node["name"])
            print node['name']
        elif node["level"] == startlevel:
            print "samelevel reached"
            return resultarray
    return resultarray

In [236]:
#find top level nodes
lvl1nodes = []
for node in newnodes:
    if node["level"] == 1:
        lvl1nodes.append(node["name"])
lvl1nodes

[u'USECASES', u'STAKEHOLDERS', u'REQUIREMENTS', u'STRUCTURE']

In [237]:
#attachChildren(0)
#for nodes that have level one, run attachChildren
for lvl1node in lvl1nodes:
    attachChildren(nodesdict[lvl1node])

attachChildren of node:  USECASES
Returning children for node  {'index': 0, 'children': [], 'name': u'USECASES', 'level': 1.0}
checking node Loading of beverages with index 1 and level 2 and startindex 0
Loading of beverages
checking node Unloading of beverages with index 2 and level 2 and startindex 0
Unloading of beverages
checking node Driving with load with index 3 and level 2 and startindex 0
Driving with load
checking node General misuse with index 4 and level 2 and startindex 0
General misuse
checking node Driving unloaded back with index 5 and level 2 and startindex 0
Driving unloaded back
checking node Transporting Pluto with index 6 and level 2 and startindex 0
Transporting Pluto
checking node Charging Pluto with index 7 and level 2 and startindex 0
Charging Pluto
checking node Disassembled transport with index 8 and level 2 and startindex 0
Disassembled transport
checking node Demonstration outside with index 9 and level 2 and startindex 0
Demonstration outside
checking node

In [243]:
#assign domains based on the lvl1node
for index, node in enumerate(newnodes):
    if node["level"] == 1:
        node["domain"] = node['name']
        otherindex = index + 1
        print index
        while newnodes[otherindex]["level"] != 1:
            newnodes[otherindex]["domain"] = node["domain"]
            print newnodes[otherindex]
            otherindex = otherindex + 1
            if otherindex == len(newnodes):
                break;

0
{'index': 1, 'domain': u'USECASES', 'children': [[]], 'name': u'Loading of beverages', 'level': 2.0}
{'index': 2, 'domain': u'USECASES', 'children': [[]], 'name': u'Unloading of beverages', 'level': 2.0}
{'index': 3, 'domain': u'USECASES', 'children': [[]], 'name': u'Driving with load', 'level': 2.0}
{'index': 4, 'domain': u'USECASES', 'children': [[]], 'name': u'General misuse', 'level': 2.0}
{'index': 5, 'domain': u'USECASES', 'children': [[]], 'name': u'Driving unloaded back', 'level': 2.0}
{'index': 6, 'domain': u'USECASES', 'children': [[]], 'name': u'Transporting Pluto', 'level': 2.0}
{'index': 7, 'domain': u'USECASES', 'children': [[]], 'name': u'Charging Pluto', 'level': 2.0}
{'index': 8, 'domain': u'USECASES', 'children': [[]], 'name': u'Disassembled transport', 'level': 2.0}
{'index': 9, 'domain': u'USECASES', 'children': [[]], 'name': u'Demonstration outside', 'level': 2.0}
{'index': 10, 'domain': u'USECASES', 'children': [[]], 'name': u'Test, Service, Repair', 'level': 2.

In [245]:
newnodes

[{'children': [[u'Loading of beverages',
    u'Unloading of beverages',
    u'Driving with load',
    u'General misuse',
    u'Driving unloaded back',
    u'Transporting Pluto',
    u'Charging Pluto',
    u'Disassembled transport',
    u'Demonstration outside',
    u'Test, Service, Repair',
    u'Error handling case',
    u'Demonstration, Show inside']],
  'domain': u'USECASES',
  'index': 0,
  'level': 1.0,
  'name': u'USECASES'},
 {'children': [[]],
  'domain': u'USECASES',
  'index': 1,
  'level': 2.0,
  'name': u'Loading of beverages'},
 {'children': [[]],
  'domain': u'USECASES',
  'index': 2,
  'level': 2.0,
  'name': u'Unloading of beverages'},
 {'children': [[]],
  'domain': u'USECASES',
  'index': 3,
  'level': 2.0,
  'name': u'Driving with load'},
 {'children': [[]],
  'domain': u'USECASES',
  'index': 4,
  'level': 2.0,
  'name': u'General misuse'},
 {'children': [[]],
  'domain': u'USECASES',
  'index': 5,
  'level': 2.0,
  'name': u'Driving unloaded back'},
 {'children': [

In [246]:
#prepare string for export to JSON
jsonstr="{  \"nodes\": ["

In [247]:
#print nodes
nodesstr=''
for node in newnodes:
    print "{\"name\": \"" + node["name"].strip() + "\",\"level\": " + str(int(round(node["level"]))).strip() + ",\"domain\": \"" + node["domain"].lower() + "\"}," 
    nodesstr = nodesstr + "{\"name\": \"" + node["name"].strip() + "\",\"level\": " + str(int(round(node["level"]))).strip() + ",\"domain\": \"" + node["domain"].lower() + "\"}," 
nodesstr

{"name": "USECASES","level": 1,"domain": "usecases"},
{"name": "Loading of beverages","level": 2,"domain": "usecases"},
{"name": "Unloading of beverages","level": 2,"domain": "usecases"},
{"name": "Driving with load","level": 2,"domain": "usecases"},
{"name": "General misuse","level": 2,"domain": "usecases"},
{"name": "Driving unloaded back","level": 2,"domain": "usecases"},
{"name": "Transporting Pluto","level": 2,"domain": "usecases"},
{"name": "Charging Pluto","level": 2,"domain": "usecases"},
{"name": "Disassembled transport","level": 2,"domain": "usecases"},
{"name": "Demonstration outside","level": 2,"domain": "usecases"},
{"name": "Test, Service, Repair","level": 2,"domain": "usecases"},
{"name": "Error handling case","level": 2,"domain": "usecases"},
{"name": "Demonstration, Show inside","level": 2,"domain": "usecases"},
{"name": "STAKEHOLDERS","level": 1,"domain": "stakeholders"},
{"name": "Operator","level": 2,"domain": "stakeholders"},
{"name": "User","level": 2,"domain": "s

u'{"name": "USECASES","level": 1,"domain": "usecases"},{"name": "Loading of beverages","level": 2,"domain": "usecases"},{"name": "Unloading of beverages","level": 2,"domain": "usecases"},{"name": "Driving with load","level": 2,"domain": "usecases"},{"name": "General misuse","level": 2,"domain": "usecases"},{"name": "Driving unloaded back","level": 2,"domain": "usecases"},{"name": "Transporting Pluto","level": 2,"domain": "usecases"},{"name": "Charging Pluto","level": 2,"domain": "usecases"},{"name": "Disassembled transport","level": 2,"domain": "usecases"},{"name": "Demonstration outside","level": 2,"domain": "usecases"},{"name": "Test, Service, Repair","level": 2,"domain": "usecases"},{"name": "Error handling case","level": 2,"domain": "usecases"},{"name": "Demonstration, Show inside","level": 2,"domain": "usecases"},{"name": "STAKEHOLDERS","level": 1,"domain": "stakeholders"},{"name": "Operator","level": 2,"domain": "stakeholders"},{"name": "User","level": 2,"domain": "stakeholders"}

In [248]:
jsonstr = jsonstr + nodesstr
#remove last comma
jsonstr = jsonstr[:-1] 
jsonstr = jsonstr + "], \"links\": ["
jsonstr

u'{  "nodes": [{"name": "USECASES","level": 1,"domain": "usecases"},{"name": "Loading of beverages","level": 2,"domain": "usecases"},{"name": "Unloading of beverages","level": 2,"domain": "usecases"},{"name": "Driving with load","level": 2,"domain": "usecases"},{"name": "General misuse","level": 2,"domain": "usecases"},{"name": "Driving unloaded back","level": 2,"domain": "usecases"},{"name": "Transporting Pluto","level": 2,"domain": "usecases"},{"name": "Charging Pluto","level": 2,"domain": "usecases"},{"name": "Disassembled transport","level": 2,"domain": "usecases"},{"name": "Demonstration outside","level": 2,"domain": "usecases"},{"name": "Test, Service, Repair","level": 2,"domain": "usecases"},{"name": "Error handling case","level": 2,"domain": "usecases"},{"name": "Demonstration, Show inside","level": 2,"domain": "usecases"},{"name": "STAKEHOLDERS","level": 1,"domain": "stakeholders"},{"name": "Operator","level": 2,"domain": "stakeholders"},{"name": "User","level": 2,"domain": "s

In [250]:
#generate links
links = []
for node in newnodes:
    #print { "source": "Day and night", "target": "Priority A"  },
    #print node["children"]
    for child in node["children"][0]:
        #print child
        link = {}
        link["source"] = str(node["name"])
        link["target"] = str(child)
        links.append(link)
        #print link
linkstr = ''
for link in links:
    linkstr = linkstr + "{ \"source\": \"%s\",\"target\": \"%s\" }," % (link["source"], link["target"])  
    print "{ \"source\": \"%s\",\"target\": \"%s\" }," % (link["source"], link["target"])  


{ "source": "USECASES","target": "Loading of beverages" },
{ "source": "USECASES","target": "Unloading of beverages" },
{ "source": "USECASES","target": "Driving with load" },
{ "source": "USECASES","target": "General misuse" },
{ "source": "USECASES","target": "Driving unloaded back" },
{ "source": "USECASES","target": "Transporting Pluto" },
{ "source": "USECASES","target": "Charging Pluto" },
{ "source": "USECASES","target": "Disassembled transport" },
{ "source": "USECASES","target": "Demonstration outside" },
{ "source": "USECASES","target": "Test, Service, Repair" },
{ "source": "USECASES","target": "Error handling case" },
{ "source": "USECASES","target": "Demonstration, Show inside" },
{ "source": "STAKEHOLDERS","target": "Operator" },
{ "source": "STAKEHOLDERS","target": "User" },
{ "source": "STAKEHOLDERS","target": "Third party" },
{ "source": "STAKEHOLDERS","target": "Builder" },
{ "source": "STAKEHOLDERS","target": "IPEK /ID" },
{ "source": "REQUIREMENTS","target": "Priori

In [251]:
jsonstr = jsonstr + linkstr
#remove last comma
jsonstr = jsonstr[:-1] 
jsonstr = jsonstr + "], \"traces\": ["
jsonstr

u'{  "nodes": [{"name": "USECASES","level": 1,"domain": "usecases"},{"name": "Loading of beverages","level": 2,"domain": "usecases"},{"name": "Unloading of beverages","level": 2,"domain": "usecases"},{"name": "Driving with load","level": 2,"domain": "usecases"},{"name": "General misuse","level": 2,"domain": "usecases"},{"name": "Driving unloaded back","level": 2,"domain": "usecases"},{"name": "Transporting Pluto","level": 2,"domain": "usecases"},{"name": "Charging Pluto","level": 2,"domain": "usecases"},{"name": "Disassembled transport","level": 2,"domain": "usecases"},{"name": "Demonstration outside","level": 2,"domain": "usecases"},{"name": "Test, Service, Repair","level": 2,"domain": "usecases"},{"name": "Error handling case","level": 2,"domain": "usecases"},{"name": "Demonstration, Show inside","level": 2,"domain": "usecases"},{"name": "STAKEHOLDERS","level": 1,"domain": "stakeholders"},{"name": "Operator","level": 2,"domain": "stakeholders"},{"name": "User","level": 2,"domain": "s

In [254]:
#print traces
tracestr = ''
for rowindex, row in enumerate(mdmdata):
    #print row
    for colindex, cell in enumerate(row):
        #if (cell > 0) and (cell != 'None') and (rowindex > 1) and (colindex > 1) and (rowindex != colindex):
        if (cell > 0) and (cell != 'None') and (rowindex > 3) and (colindex > 3) and (rowindex != colindex):
            #print cell, rowindex-2, colindex-1
            tracestr = tracestr + "{ \"nodeids\":\"" + str(rowindex-4) + "," + str(colindex-3) + "\" },"
            print "{ \"nodeids\":\"" + str(rowindex-4) + "," + str(colindex-3) + "\" },"
            #print newnodes[rowindex-4]["name"], newnodes[colindex-3]["name"] 

{ "nodeids":"14,1" },
{ "nodeids":"14,5" },
{ "nodeids":"14,6" },
{ "nodeids":"14,7" },
{ "nodeids":"14,10" },
{ "nodeids":"14,11" },
{ "nodeids":"15,2" },
{ "nodeids":"15,3" },
{ "nodeids":"16,4" },
{ "nodeids":"17,8" },
{ "nodeids":"17,10" },
{ "nodeids":"17,11" },
{ "nodeids":"18,9" },
{ "nodeids":"18,12" },
{ "nodeids":"21,3" },
{ "nodeids":"21,5" },
{ "nodeids":"22,3" },
{ "nodeids":"22,5" },
{ "nodeids":"23,3" },
{ "nodeids":"23,5" },
{ "nodeids":"24,3" },
{ "nodeids":"24,5" },
{ "nodeids":"25,3" },
{ "nodeids":"25,5" },
{ "nodeids":"25,9" },
{ "nodeids":"26,7" },
{ "nodeids":"27,3" },
{ "nodeids":"27,4" },
{ "nodeids":"28,1" },
{ "nodeids":"28,2" },
{ "nodeids":"28,3" },
{ "nodeids":"29,2" },
{ "nodeids":"29,4" },
{ "nodeids":"29,5" },
{ "nodeids":"30,3" },
{ "nodeids":"31,3" },
{ "nodeids":"32,1" },
{ "nodeids":"32,3" },
{ "nodeids":"33,3" },
{ "nodeids":"33,5" },
{ "nodeids":"34,3" },
{ "nodeids":"34,5" },
{ "nodeids":"35,3" },
{ "nodeids":"35,5" },
{ "nodeids":"36,7" },
{ "no

In [255]:
jsonstr = jsonstr + tracestr
#remove last comma
jsonstr = jsonstr[:-1] 
jsonstr = jsonstr + "] }"
jsonstr

u'{  "nodes": [{"name": "USECASES","level": 1,"domain": "usecases"},{"name": "Loading of beverages","level": 2,"domain": "usecases"},{"name": "Unloading of beverages","level": 2,"domain": "usecases"},{"name": "Driving with load","level": 2,"domain": "usecases"},{"name": "General misuse","level": 2,"domain": "usecases"},{"name": "Driving unloaded back","level": 2,"domain": "usecases"},{"name": "Transporting Pluto","level": 2,"domain": "usecases"},{"name": "Charging Pluto","level": 2,"domain": "usecases"},{"name": "Disassembled transport","level": 2,"domain": "usecases"},{"name": "Demonstration outside","level": 2,"domain": "usecases"},{"name": "Test, Service, Repair","level": 2,"domain": "usecases"},{"name": "Error handling case","level": 2,"domain": "usecases"},{"name": "Demonstration, Show inside","level": 2,"domain": "usecases"},{"name": "STAKEHOLDERS","level": 1,"domain": "stakeholders"},{"name": "Operator","level": 2,"domain": "stakeholders"},{"name": "User","level": 2,"domain": "s

In [259]:
with open('graph_july3_from_matrix.json', 'w') as outfile:  
    outfile.write(jsonstr)

## One domain

In [ ]:
sht = wb.sheets['Nodes']

In [87]:
nodestwo = sht.range('A1:C20').value
nodestwo

[[u'A0', u'Information', 1.0],
 [u'A1', u'Business', 2.0],
 [u'A2', u'- Client', 3.0],
 [u'A3', u'- Requirements', 3.0],
 [u'A4', u'Configuration', 2.0],
 [u'A5', u'- Plant Type', 3.0],
 [u'A6', u'- Plant Size', 3.0],
 [u'A7', u'- Dimensions', 3.0],
 [u'A8', u'- Intended product/powder', 3.0],
 [u'A9', u'- Interface with building', 3.0],
 [u'A10', u'- BOM', 3.0],
 [u'A11', u'Operation', 2.0],
 [u'A12', u'- Process Parameters', 3.0],
 [u'A13', u'- Realized processdata', 3.0],
 [u'A14', u'Project', 2.0],
 [u'A15', u'- Planning and Scheduling', 3.0],
 [u'A16', u'- Resources (e.g. hours)', 3.0],
 [u'A17', u'- Pricing', 3.0],
 [u'A18', u'- Procurement', 3.0],
 [u'A19', u'- Cost', 3.0]]

In [89]:
for node in nodestwo:
    print "{\"name\": \"" + node[1].strip() + "\",\"level\": " + str(int(round(node[2]))).strip() + ",\"domain\": \"usecases\"}," 

{"name": "Information","level": 1,"domain": "usecases"},
{"name": "Business","level": 2,"domain": "usecases"},
{"name": "- Client","level": 3,"domain": "usecases"},
{"name": "- Requirements","level": 3,"domain": "usecases"},
{"name": "Configuration","level": 2,"domain": "usecases"},
{"name": "- Plant Type","level": 3,"domain": "usecases"},
{"name": "- Plant Size","level": 3,"domain": "usecases"},
{"name": "- Dimensions","level": 3,"domain": "usecases"},
{"name": "- Intended product/powder","level": 3,"domain": "usecases"},
{"name": "- Interface with building","level": 3,"domain": "usecases"},
{"name": "- BOM","level": 3,"domain": "usecases"},
{"name": "Operation","level": 2,"domain": "usecases"},
{"name": "- Process Parameters","level": 3,"domain": "usecases"},
{"name": "- Realized processdata","level": 3,"domain": "usecases"},
{"name": "Project","level": 2,"domain": "usecases"},
{"name": "- Planning and Scheduling","level": 3,"domain": "usecases"},
{"name": "- Resources (e.g. hours)",

In [ ]:
links = {}
source = 

In [92]:
nodesdict = {}
for index, node in enumerate(nodestwo):
    nodesdict[node[0]] = index
nodesdict

{u'A0': 0,
 u'A1': 1,
 u'A10': 10,
 u'A11': 11,
 u'A12': 12,
 u'A13': 13,
 u'A14': 14,
 u'A15': 15,
 u'A16': 16,
 u'A17': 17,
 u'A18': 18,
 u'A19': 19,
 u'A2': 2,
 u'A3': 3,
 u'A4': 4,
 u'A5': 5,
 u'A6': 6,
 u'A7': 7,
 u'A8': 8,
 u'A9': 9}

In [93]:
celllist = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
#for index,topcells in enumerate(columns, start = 1):
for index in range(7,20):
    cellstr = celllist[index].upper() + "2:" + celllist[index].upper() + "20"
    source = celllist[index].upper() + "1"
    #print cellstr
    sourcecol = sht.range(cellstr).value
    #print sourcecol
    for cell in sourcecol:
        if (cell != -2146826259) and (sht.range(source).value != cell):
            #print sht.range(source).value, cell
            #print nodesdict[sht.range(source).value], nodesdict[cell]
            print "{ \"nodeids\":\"" + str(nodesdict[sht.range(source).value]) + "," + str(nodesdict[cell]) + "\" },"

{ "nodeids":"1,2" },
{ "nodeids":"1,3" },
{ "nodeids":"1,15" },
{ "nodeids":"2,3" },
{ "nodeids":"2,15" },
{ "nodeids":"3,4" },
{ "nodeids":"3,5" },
{ "nodeids":"3,6" },
{ "nodeids":"3,7" },
{ "nodeids":"3,10" },
{ "nodeids":"3,11" },
{ "nodeids":"3,12" },
{ "nodeids":"3,13" },
{ "nodeids":"3,15" },
{ "nodeids":"3,18" },
{ "nodeids":"3,19" },
{ "nodeids":"4,5" },
{ "nodeids":"4,6" },
{ "nodeids":"4,7" },
{ "nodeids":"4,9" },
{ "nodeids":"4,10" },
{ "nodeids":"4,15" },
{ "nodeids":"4,18" },
{ "nodeids":"4,19" },
{ "nodeids":"5,6" },
{ "nodeids":"5,7" },
{ "nodeids":"5,10" },
{ "nodeids":"5,15" },
{ "nodeids":"5,16" },
{ "nodeids":"5,17" },
{ "nodeids":"5,18" },
{ "nodeids":"5,19" },
{ "nodeids":"6,7" },
{ "nodeids":"6,10" },
{ "nodeids":"6,15" },
{ "nodeids":"6,16" },
{ "nodeids":"6,17" },
{ "nodeids":"6,18" },
{ "nodeids":"6,19" },
{ "nodeids":"7,10" },
{ "nodeids":"7,18" },
{ "nodeids":"7,19" },
{ "nodeids":"8,16" },
{ "nodeids":"8,18" },
{ "nodeids":"8,19" },
{ "nodeids":"9,18" },
{ 

In [20]:
for cell in column:
    if source != cell:
        print source, cell

A1 A2
A1 A3
A1 -2146826259
A1 -2146826259
A1 -2146826259
A1 -2146826259
A1 -2146826259
A1 -2146826259
A1 -2146826259
A1 -2146826259
A1 -2146826259
A1 -2146826259
A1 -2146826259
A1 A15
A1 -2146826259
A1 -2146826259
A1 -2146826259
A1 -2146826259
